In [1]:
import sys
sys.path.insert(0,"/home/shackste/galaxy-generator/python_modules/")

import matplotlib.pyplot as plt

plt.style.use('dark_background')

from pdb import set_trace

# Classifier

In [2]:
from image_classifier import ImageClassifier
from accuracy_measures import measure_accuracy_classifier
from loss import get_sample_variance, loss_sample_variance


!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under galaxy-generator/data/
the results will be placed there, too.



# Training

In [3]:
import torch 
from torch.optim import SGD

from dataset import MakeDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = SGD
optimizer_kwargs = {"nesterov":True, "momentum":0.9}
learning_rate_init = 0.04
gamma = 0.995 # learning rate decay factor
sample_variance_threshold = 0.002
seed_parameter = 7

weight_loss_sample_variance = 0 # 10.

epochs = 1000 #500 #100  #  433 epochs done by Dielemann et al. 2015

batch_size = 32

epochs = 500

make_data_loader = MakeDataLoader()
classifier = ImageClassifier(seed=seed_parameter,
                             gamma=gamma,
                             sample_variance_threshold=sample_variance_threshold, 
                             optimizer=optimizer,
                             optimizer_kwargs=optimizer_kwargs, 
                             learning_rate_init=learning_rate_init,
                             weight_loss_sample_variance=weight_loss_sample_variance
                            ).to(device)

classifier.load()
classifier.use_label_hierarchy()

schedule = {
    # epoch : performed change
    2 : classifier.use_label_hierarchy,
}



In [4]:
print(device)
torch.cuda.is_available(), classifier.make_labels_hierarchical, classifier.considered_groups
#data_loader_train.num_workers


cuda


(True, True, {1, 2, 3, 4, 5, 6, 7, 9})

In [5]:

epoch = 1

while epoch < epochs+1:
    ## adjust training according to schedule
    if classifier.epoch in schedule.keys():
        schedule[classifier.epoch]()
    data_loader_train = make_data_loader.get_data_loader_train(batch_size=batch_size) #, num_workers=4)
    data_loader_valid = make_data_loader.get_data_loader_valid(batch_size=batch_size) #, num_workers=4)
    classifier.train_epoch(data_loader_train, data_loader_valid)
    classifier.plot_losses()
    classifier.plot_accuracy()
    classifier.plot_test_accuracy()
    classifier.plot_sample_variances()
    
        


epoch 0:   0%|          | 0/1731 [00:00<?, ?it/s]/home/shackste/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
epoch 0:   2%|▏         | 37/1731 [00:12<09:11,  3.07it/s]


KeyboardInterrupt: 

In [13]:
data_loader_train = make_data_loader.get_data_loader_train(batch_size=batch_size) #, num_workers=4)

torch.autograd.detect_anomaly?

for image, label in data_loader_train:
    print(image.min())
    break


tensor(0., device='cuda:0')


Init signature: torch.autograd.detect_anomaly() -> None
Docstring:     
Context-manager that enable anomaly detection for the autograd engine.

This does two things:

- Running the forward pass with detection enabled will allow the backward
  pass to print the traceback of the forward operation that created the failing
  backward function.
- Any backward computation that generate "nan" value will raise an error.

.. warning::
    This mode should be enabled only for debugging as the different tests
    will slow down your program execution.

Example:

    >>> import torch
    >>> from torch import autograd
    >>> class MyFunc(autograd.Function):
    ...     @staticmethod
    ...     def forward(ctx, inp):
    ...         return inp.clone()
    ...     @staticmethod
    ...     def backward(ctx, gO):
    ...         # Error during the backward pass
    ...         raise RuntimeError("Some error in backward")
    ...         return gO.clone()
    >>> def run_fn(a):
    ...     out = MyF

In [ ]:
for label in labels_pred:
    print(label)

In [ ]:
for group, accuracy in accuracies.items():
    if not group in considered_groups:
        continue
    plt.title("accuracy")
    plot_losses(accuracy, test_losses_steps, iteration, label=f"Q{group}")
    plt.ylim(-0.1,1.1)
    plt.show()

In [ ]:
N_aug = 16
N_batch = 3

with torch.no_grad():
    for images, labels in make_data_loader.get_data_loader_train(batch_size=batch_size, num_workers=4):
        images = images.to(device)
        labels = labels.to(device)
        break


imgs = classifier.augment(images)
shp = imgs.shape
print(imgs.shape, images.shape)
imgs = imgs.reshape(N_aug, N_batch,*shp[1:])
imgs = imgs.permute(1,0,2,3,4)
imgs = imgs.reshape(N_aug*N_batch,*shp[1:])
print(imgs.shape)

#for image, l in zip(imgs, classifier(images)):
#    print(l)
for image in imgs:
    plt.imshow(image.permute(1,2,0).cpu())
    plt.show()


In [ ]:
for g in classifier.optimizer.param_groups:
    print(g["lr"])

In [ ]:
with torch.no_grad():
    data_loader_test = make_data_loader.get_data_loader_train(batch_size=batch_size_test, num_workers=4)
    loss = 0
    accs = Counter({group:0 for group in range(1,12)})
    for N_test, (images_test, labels_test) in enumerate(data_loader_test):
        images_test = images_test.to(device)
        labels_test = labels_test.to(device)
        labels_pred = classifier(images_test)
        loss += mse(labels_pred, labels_test).item()
        accs_tmp = measure_accuracy_classifier(labels_pred, labels_test)
        accs.update(Counter(accs_tmp))
        if N_test == N_batches_test:
            break                        
        break
print(accs)
print(accs[1])
for i in range(len(labels_test)):   
    print("predic", labels_pred[i,:3])
    print("target", labels_test[i,:3])
    print("diff  ", labels_test[i,:3] - labels_pred[i,:3])
    
    print()
    

In [ ]:
print(iteration, len(test_losses), test_losses_steps)
plot_losses(losses, losses_steps, iteration, label="training")
plot_losses(test_losses, test_losses_steps, iteration, label="test")
plt.show()
        

In [ ]:
ll = torch.rand(2,37)
make_galaxy_labels_hierarchical(ll)

## Interpretation of sample variance

In [ ]:
N_batch, N_feat = 32, 3
m = torch.rand(1,N_feat)
differences = [0.0001,0.001, 0.01, 0.1, 1]
var = []
for d in differences:
    sample = m + d*torch.randn(N_batch,N_feat)
    var.append(get_sample_variance(sample))
plt.loglog(differences, var)
plt.xlabel("average difference")
plt.ylabel("sample variance")

the sample variance is about the squared average difference: sample variance of $10^{-4}$ refers to mean difference of 1%